In [21]:
import codecs
import os
import sys
import zipfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

In [22]:
(train_ds, test_ds), metadata = tfds.load('colorectal_histology',split=['train[:80%]','train[80%:]'],with_info=True,
        as_supervised=True)

In [23]:
class CNN(tf.keras.Sequential):
    def __init__(self):
        super(CNN, self).__init__(layers=[
            tf.keras.layers.Rescaling(1./255, input_shape=(150,150,3)),
            tf.keras.layers.Conv2D(32, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(128, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(8, activation= 'softmax')
        ])

In [24]:
model = CNN()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [25]:
model.compile(optimizer=optimizer, loss=criterion, metrics=['accuracy'])

In [26]:
model.summary()

Model: "cnn_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 146, 146, 32)      2432      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 69, 69, 64)        51264     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 34, 34, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 30, 30, 128)       204928

In [27]:
15*15*32

7200

In [28]:
model.fit(
  train_ds.batch(32),
  epochs=1
)

125/125 [==============================] - 97s 771ms/step - loss: 1.3290 - accuracy: 0.4325


In [30]:
model.evaluate(test_ds.batch(32), verbose=2)

c:\miniconda\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


32/32 - 7s - loss: 0.9965 - accuracy: 0.4810 - 7s/epoch - 212ms/step


[0.9965391755104065, 0.48100000619888306]